.

<p align="center">
  <a href="https://github.com/somaSystems/HyperThelia" target="_blank">
    <img src="https://github.com/somaSystems/HyperThelia/blob/main/Hyperthelia_project/notebooks/lib/hyper_thelia.png?raw=1" alt="HyperThelia Logo" width="400"/>
  </a>
</p>

<p align="center">
  <strong>Segment. Track. Measure. <br> 3D Cell Analysis in Google Colab</strong>
</p>

<a name="top"></a>
# Hyperthelia Pipeline

**Table of Contents**
- [PART 1: Setup](#part-1-setup)
- [PART 2: Segmentation](#part-2-segmentation)
- [PART 3: Tracking](#part-3-tracking)
- [PART 4: Measurement and Export](#part-4-measurement-and-export)

To run everything, use **Runtime > Run all** from the menu.


## PART 1: Setup
[Back to Top](#hyperthelia-pipeline)


In [ ]:
# ========================
# USER CONFIGURATIONS START
# ========================
from pathlib import Path

# OPTIONAL: Mount Google Drive, to save and access data on your drive.
# from google.colab import drive
# drive.mount('/content/drive')

CLONE_DIR = Path("/content/HyperThelia")  # CLONE_DIR: Where to clone the project repo
# CLONE_DIR = Path("/content/drive/MyDrive/HyperThelia")  # Use Google Drive

# RAW_DIR = None # RAW_DIR: Leave as None to show an upload box.
RAW_DIR = Path("/content/HyperThelia/Hyperthelia_project/data_demo")  # Or use built-in: Path("raw_data")

RAW_INTENSITY_DIR = Path("/content/HyperThelia/Hyperthelia_project/raw_intensity")


# BASE_DIR: Optional override for saving results (default = CLONE_DIR)
# BASE_DIR = Path("/content/drive/MyDrive/HyperThelia_outputs")

In [ ]:
# === REPO CLONE LOGIC ===
import shutil
import subprocess

REPO_URL = "https://github.com/somaSystems/HyperThelia.git"
CLONE_BRANCH = "hyperThelia"
force_clone = True

if CLONE_DIR.exists() and force_clone:
    print(f"Force-cloning: removing {CLONE_DIR}")
    shutil.rmtree(CLONE_DIR)

if not CLONE_DIR.exists():
    print(f"Cloning HyperThelia ({CLONE_BRANCH}) to {CLONE_DIR}...")
    subprocess.run(["git", "clone", "--branch", CLONE_BRANCH, REPO_URL, str(CLONE_DIR)], check=True)
else:
    print("Repo already exists. Set force_clone=True to re-clone.")

import sys
lib_path = CLONE_DIR / "Hyperthelia_project" / "notebooks" / "lib"
if str(lib_path) not in sys.path:
    sys.path.insert(0, str(lib_path))

# === RAW DATA UPLOAD INTERFACE ===
from UsersDataDemo import choose_upload_method
BASE_PROJECT_DIR = CLONE_DIR / "Hyperthelia_project"
if RAW_DIR is None:
    RAW_DIR = choose_upload_method(BASE_PROJECT_DIR)

In [ ]:
# Install latest Cellpose from GitHub (includes SAM support)
!pip install git+https://github.com/mouseland/cellpose.git

In [ ]:
# === SESSION SETUP LOCATE LIB AND FUNCTIONS ===
from setup_functions import setup_hyperthelia_project
from setup_functions import setup_project_io

BASE_PROJECT_DIR, RAW_DIR, OUTPUTS_DIR = setup_hyperthelia_project(
    clone_dir=CLONE_DIR,
    raw_dir=RAW_DIR,
    base_dir=globals().get("BASE_DIR", None)
)

# === SESSION SETUP IMPORTS AND FUNCTIONS ===
import segmentation
import importlib
importlib.reload(segmentation)

# Use the RAW_DIR defined earlier (from user upload or demo)
RAW_DIR, OUTPUTS_DIR = setup_project_io(BASE_PROJECT_DIR, raw_dir=RAW_DIR)

# SETUP LOGGER, CHECK GPU, and CELLPOSE MODEL
from segmentation import setup_cellpose_model
model = setup_cellpose_model(gpu=True)



In [ ]:
# === LOOK FOR EXPERIMENTS IN RAW_DIR ===
from segmentation import print_experiment_summary
print_experiment_summary(RAW_DIR)


<a name="part-2-segmentation"></a>
## PART 2: Segmentation

[Back to Top](#top)

In [ ]:
from segmentation import run_segmentation_pipeline

# === SEGMENTATION PARAMETERS ===
Z_AXIS = 0
CHANNEL_AXIS = None
BATCH_SIZE = 32
DO_3D = False
STITCH_THRESHOLD = 0.5

# === RUN PIPELINE ===
run_segmentation_pipeline(
    RAW_DIR,
    OUTPUTS_DIR,
    model,
    z_axis=Z_AXIS,
    channel_axis=CHANNEL_AXIS,
    batch_size=BATCH_SIZE,
    do_3D=DO_3D,
    stitch_threshold=STITCH_THRESHOLD
)


In [ ]:
from measurevisualise import interactive_segmentation_viewer
interactive_segmentation_viewer(BASE_PROJECT_DIR / "outputs")


<a name="part-3-tracking"></a>
## PART 3: Tracking

[Back to Top](#top)

In [ ]:
import tracking
import propagation
import visualisation

import importlib
# importlib.reload(tracking)
# importlib.reload(propagation)
# importlib.reload(visualisation)

In [ ]:
# === AUTO CONFIGURATION ===

# Tracking parameters (editable by user)
XY_UM = 0.325
Z_UM = 1.0
MAX_DIST_UM = 10
MIN_VOLUME = 5000
MAX_VOLUME = 80000
EDGE_MARGIN = 1
TRACKING_MODE = "nearest"

In [ ]:
# === STEP 1: VISUALISE PRE-TRACKING ===
from visualisation import plot_volume_histogram_for_experiment, view_segmentation_slice_with_boundaries, get_segmented_tiffs_by_experiment

experiments = get_segmented_tiffs_by_experiment(OUTPUTS_DIR)

plot_volume_histogram_for_experiment(exp_index=0,
                                                   experiments_dict=experiments,
                                                   min_volume=MIN_VOLUME,
                                                   max_volume=MAX_VOLUME)

view_segmentation_slice_with_boundaries(exp_index=0,
                                        experiments_dict=experiments,
                                        time_index=0,
                                        z_slice=18,
                                        edge_margin=EDGE_MARGIN)



In [ ]:
# === STEP 2: RUN TRACKING ===
tracking.run_tracking_pipeline(
    output_base_dir=OUTPUTS_DIR,
    xy_um=XY_UM,
    z_um=Z_UM,
    max_dist_um=MAX_DIST_UM,
    min_volume=MIN_VOLUME,
    max_volume=MAX_VOLUME,
    edge_margin=EDGE_MARGIN,
    tracking_mode=TRACKING_MODE
)

In [ ]:
# === STEP 3: PROPAGATE LABELS ===
propagation.run_propagation_pipeline(output_base_dir=OUTPUTS_DIR)

In [ ]:
# === STEP 4: VISUALISE POST-TRACKING ===
visualisation.plot_tracked_centroids_xy_by_index(exp_index=0, output_base_dir=OUTPUTS_DIR)
visualisation.preview_propagated_labels_zslice(exp_index=0, z_slice=13, output_base_dir=OUTPUTS_DIR)

In [ ]:
# === PART 5: Create Regions (cytoplasm & membrane) ===
import importlib, regions
importlib.reload(regions)

# Create eroded cytoplasm, then membrane = full − cytoplasm
regions.create_cytoplasm_and_membrane(
    output_base_dir=OUTPUTS_DIR,
    erosion_px=4,               # shrink cells by 4 px
    cytoplasm_region="cytoplasm",
    membrane_region="membrane"
)


<a name="part-4-measurement-and-export"></a>
## PART 4: Measurement and Export

[Back to Top](#top)


In [ ]:
import importlib
import measurement
import measurevisualise
importlib.reload(measurevisualise)
importlib.reload(measurement)

In [ ]:
# ===  USER TOGGLES FOR MEASURES ===
is_tracked = True
compute_surface = True
enable_intensity_measurement = True
intensity_channel_mode = "folder"
force = False
# RAW_INTENSITY_DIR = BASE_PROJECT_DIR / "raw_intensity"
EXPORT_DIR = BASE_PROJECT_DIR / "image_exports"


In [ ]:
# ===  LIST AVAILABLE EXPERIMENTS ===
experiment_data = measurement.discover_experiments(OUTPUTS_DIR, is_tracked=is_tracked)
measurement.summarise_experiment_data(experiment_data)

In [ ]:

measurement.run_all_measurements_for_all_sets(
    outputs_dir=OUTPUTS_DIR,
    is_tracked=is_tracked,
    compute_surface=compute_surface,
    enable_intensity_measurement=enable_intensity_measurement,
    intensity_dir=RAW_INTENSITY_DIR,
    force=force,
    measure_mode="all"
)



In [ ]:
import pandas as pd
# Load the CSV and show all available measurement columns
csv_path = measurevisualise.list_available_measurement_csvs(BASE_PROJECT_DIR, return_first=True)
df = pd.read_csv(csv_path)

print("Available measurement columns:")
print(df.columns.tolist())


In [ ]:
from measurevisualise import interactive_measurement_viewer
interactive_measurement_viewer(BASE_PROJECT_DIR / "outputs")


In [ ]:
#  Scripted mode: displaying specified measurement view

from measurevisualise import interactive_measurement_viewer

interactive_measurement_viewer(
    output_base_dir=BASE_PROJECT_DIR / "outputs",
    csv_path="outputs_demothelia/measured/regionprops_demothelia_tracked_2D.csv",
    timepoint=1,
    z=12,
    value_column="area_2D"
)



In [ ]:
from measurevisualise import export_measurement_values_as_tiff

csv_path = BASE_PROJECT_DIR / "outputs" / "outputs_demothelia" / "measured" / "regionprops_demothelia_tracked_3D.csv"
output_base_dir = BASE_PROJECT_DIR / "outputs"

export_measurement_values_as_tiff(
    csv_path=csv_path,
    base_dir=output_base_dir,
    timepoint=1,
    value_column="area_voxels",
)


In [ ]:
from datetime import datetime
print(f"Finished run at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")